In [1]:
import fcalc

import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score

Reading data

In [4]:
all_data = pd.read_csv('creditcard_2023.csv')
all_data.drop(columns = ['id'], inplace = True)
# all_data, tmp = train_test_split(all_data, shuffle=True, random_state=42, stratify=all_data['Class'], train_size=0.005)

Binarizing data

In [3]:
num_cut = 5
X, y = all_data.drop(columns = 'Class'), all_data['Class']
for column in X.columns:
    X[column] = pd.qcut(X[column], num_cut)

In [4]:
X = pd.get_dummies(X)

Assesing performance

In [5]:
def cross_val_score(X, y, n_splits, scoring):
    skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)
    scores = np.zeros((len(scoring), skf.get_n_splits()))

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        bin_cls = fcalc.classifier.BinarizedBinaryClassifier(X_train.values, y_train.values)
        bin_cls.predict(X_test.values)

        # we always classify non prediction as false prediction
        tmp_predict = np.copy(bin_cls.predictions)
        tmp_predict[tmp_predict == -1] = np.abs(y_test.values[tmp_predict == -1] - 1)

        for j, score in enumerate(scoring):
            scores[j, i] = score(y_test.values, tmp_predict)
    
    return scores.mean(1).round(3)


In [ ]:
cross_val_score(X, y, 5, [accuracy_score, f1_score])

array([0.787, 0.751])